In [ ]:
import struct
def raw_imread(filename):
    fp = open(filename, 'rb')
    w, = struct.unpack("<i", fp.read(4))
    h, = struct.unpack("<i", fp.read(4))
    print(w,h)
    a = [[struct.unpack("<H", fp.read(2))[0] for i in range(w)] for i in range(h)]
    return a 

In [ ]:
from matplotlib import pyplot as plt 
import cv2 
img = raw_imread("lung.raw")
plt.imshow([[[i] for i in j] for j in img], cmap="gray")
plt.show()

In [ ]:
import numpy as np
img = np.array(img, dtype='uint16')
plt.imshow(img, cmap='gray')

In [ ]:
dft = np.fft.fftshift(cv2.dft(np.float32(img), flags = cv2.DFT_COMPLEX_OUTPUT))
plt.figure(figsize=(20,10))
dft_a, dft_p = cv2.cartToPolar(dft[:,:,0], dft[:,:,1])
plt.imshow(20*np.log(dft_a), cmap="gray")

In [ ]:
h, w = img.shape

In [ ]:
mask = np.zeros((h, w), np.float32)
h2, w2 = h//2, w//2
mask[h2-30:h2+30, w2-20:w2+20] = 1
plt.imshow(mask[:,:])

In [ ]:
dft_a *= mask
plt.imshow(20*np.log(dft_a))

In [ ]:
spec = np.zeros((h, w, 2), np.float32)
p, q = cv2.polarToCart(dft_a, dft_p)
spec[:,:,0] = p
spec[:,:,1] = q
print(spec.shape)

In [ ]:
ifft = cv2.idft(np.fft.ifftshift(spec))
plt.figure(figsize=(20,10))
img_blur = cv2.magnitude(ifft[:,:,0], ifft[:,:,1])
img_blur = img_blur / 1776 / 966
print(np.max(img_blur))
plt.imshow(img_blur, cmap='gray')

In [ ]:
img_l = img_blur
img_h = img - img_l 
print(np.max(img_l), np.max(img_h), np.max(img))
plt.figure(figsize=(25,10))
plt.subplot(131)
plt.imshow(img, cmap='gray')
plt.subplot(132)
plt.imshow(img_h, cmap='gray')
plt.subplot(133)
plt.imshow(img + 1.0 * img_h, cmap='gray')

In [ ]:
img_g = np.float32([[max(min(x, 500), 0) for x in u] for u in img])

In [ ]:
g1 = cv2.getGaussianKernel(11, 0.5)
g = g1 * g1.T
dst = cv2.filter2D(img, -1, g)

In [ ]:
g2 = cv2.getGaussianKernel(11, 1)
gg = g2 * g2.T
dst2 = cv2.filter2D(img, -1, gg)

In [ ]:
plt.figure(figsize=(40,40))
img_h = img + (-1) * dst + (+0.5) * (img + (-1) * dst2)
plt.subplot(121)
plt.imshow(img_g, cmap='gray')
plt.subplot(122)
img_usm = img + 10 * img_h
print(np.min(dst), np.max(dst))
print(np.min(img), np.max(img))
print(np.min(img_h), np.max(img_h))
img_usm_g = np.float32([[max(min(x, 500), 0) for x in u] for u in img_usm])
plt.imshow(img_usm_g, cmap='gray')

In [ ]:
img = img[:700, :700]

In [ ]:
def f(sigma):
    g1 = cv2.getGaussianKernel(11, sigma)
    g = g1 * g1.T
    dst = cv2.filter2D(img, -1, g)
    a = dst
    return a

def gf(sig0, sigma, alpha):
    g1 = cv2.getGaussianKernel(11, sigma)
    g = g1 * g1.T
    img_t = f(sig0)
    dst = cv2.filter2D(img_t, -1, g)
    a = img_t + (-1)*dst
    a = alpha * a + img_t
    a = a.clip(min=0, max=500)
    return a

plt.figure(figsize=(30,30))
plt.imshow(gf(0.8, 1, 10), cmap='gray')


In [ ]:
simg = img[500:550, 550:600]
plt.imshow(simg, cmap='gray')

In [ ]:
img=simg
plt.imshow(f(1.5))

In [ ]:
def f(sigma):
    g1 = cv2.getGaussianKernel(19, sigma)
    g = g1 * g1.T
    dst = cv2.filter2D(img, -1, g)
    a = dst
    return a

img_b = f(1.6)
img_o = cv2.Laplacian(img_b,cv2.CV_32F, ksize = 3)
img_l = img + (-5) * img_o
plt.figure(figsize=(50,50))
plt.subplot(121)
plt.imshow(img.clip(0,4200), cmap='gray')
plt.subplot(122)
plt.imshow(img_l.clip(0,4200), cmap='gray')